In [1]:
from coffea.processor.servicex import DataSource, Analysis
from coffea.processor.servicex import DaskExecutor, LocalExecutor 
from func_adl import ObjectStream

import pandas as pd
import servicex as sx
from func_adl_servicex import ServiceXSourceUpROOT
dataset_name = ["root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root"]
sx_dataset = sx.ServiceXDataset(dataset_name, backend_name="open_uproot")
src = ServiceXSourceUpROOT(sx_dataset, "Events")

In [2]:
class Processor(Analysis):
    def process(events):
        output = self.accumulator.identity()
        
        dataset = events.metadata["dataset"]
        
        muons = events.Muon
        MET = events.MET.pt
        
        output['cutflow']['all events'] += ak.size(muons, axis=0)
        output['cutflow']['all muons'] += ak.sum(ak.num(muons, axis=1))
        
        # Get all combinations of muon pairs in every event.
        dimuons = ak.combinations(muons, 2, fields=['i0', 'i1'])
        
        # Check that pairs have opposite charge.
        opposites = (dimuons['i0'].charge != dimuons['i1'].charge)
        
        # Get only muons with mass between 60 and 120. Add the dimuon p4's, take mass.
        limits = ((dimuons['i0'] + dimuons['i1']).mass >= 60) & ((dimuons['i0'] + dimuons['i1']).mass < 120)
        
        # Mask the dimuons with the opposites and the limits to get dimuons with opposite charge and mass between 60 and 120 GeV.
        good_dimuons = dimuons[opposites & limits]
        
        # Mask the MET to get it only if an associated dimuon pair meeting the conditions exists.
        good_MET = MET[ak.num(good_dimuons, axis=1) >= 1]
        
        output['cutflow']['final events'] += ak.size(good_MET, axis=0)
        
        output['MET'].fill(dataset=dataset, MET=good_MET)
        
        return output

In [3]:
def sx_query(source: ObjectStream) -> ObjectStream:
    return source.Select(lambda e:
        {
            'MET_pt': e['MET_pt']
        })
df = DataSource(query=sx_query, metadata={'dataset': 'root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root'}, datasets=sx_dataset)

In [4]:
from dask.distributed import Client

analysis = Processor()
executor = LocalExecutor()
#executor = DaskExecutor(client_addr="tls://matousadamec-40gmail-2ecom.dask.coffea.casa:8786")

async def run_updates_stream(accumulator_stream):
  global first
  count = 0
  async for coffea_info in accumulator_stream:
    count += 1
    print(count, coffea_info)
  return coffea_info
result = await run_updates_stream(executor.execute(analysis, df))

AttributeError: 'function' object has no attribute 'query_ast'